使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第四章第二節 ダクト式セントラル空調機）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [4]:
import nbimporter
import numpy as np
from scipy import optimize

from s04_02_central_heating_system_with_duct import get_season_array_d_t
from s04_08_a_heatpump_cycle import calc_e_ref_H_th
from s11_01_external_condition import load_outdoor, get_Theta_ex


Importing Jupyter notebook from s04_08_a_heatpump_cycle.ipynb


ImportError: cannot import name 'calc_e_ref_H_th'

## 付録 A ダクト式セントラル空調機のエネルギー消費量

### A.1 記号及び単位

省略

### A.2 消費電力量

$$\large
E_{E,H,d,t}=E_{E,comp,H,d,t}+E_{E,fan,H,d,t}
$$

$$\large
E_{E,C,d,t}=E_{E,comp,C,d,t}+E_{E,fan,C,d,t}
$$

$E_{E,C,d,t}$：日付$d$の時刻$t$における 1 時間当たりの冷房時の消費電力量(kWh/h)  
$E_{E,H,d,t}$：日付$d$の時刻$t$における 1 時間当たりの暖房時の消費電力量(kWh/h)  
$E_{E,comp,H,d,t}$：日付$d$の時刻$t$における 1 時間当たりの冷房時の圧縮機の消費電力量(kWh/h)  
$E_{E,comp,H,d,t}$：日付$d$の時刻$t$における 1 時間当たりの暖房時の圧縮機の消費電力量(kWh/h)  
$E_{E,fan,H,d,t}$：日付$d$の時刻$t$における 1 時間当たりの送風機の消費電力量のうちの冷房設備への付加分(kWh/h)  
$E_{E,fan,H,d,t}$：日付$d$の時刻$t$における 1 時間当たりのの送風機の消費電力量のうちの暖房設備への付加分(kWh/h)  


In [81]:
# 日付dの時刻tにおける1時間当たりの暖房時の消費電力量（kWh/h）(1)
def calc_E_E_H_d_t(Theta_hs_out_d_t, Theta_hs_in_d_t, V_hs_supply_d_t, V_hs_vent_d_t, C_df_H_d_t,
           q_hs_rtd_H, V_hs_dsgn_H, P_hs_mid_H, P_hs_rtd_H, P_fan_rtd_H, P_fan_mid_H, q_hs_min_H, q_hs_mid_H,
           V_fan_rtd_H, V_fan_mid_H, EquipmentSpec, region):

    # 外気条件
    outdoor = load_outdoor()
    Theta_ex_d_t = get_Theta_ex(region, outdoor)

    # (3)
    q_hs_H_d_t = get_q_hs_H_d_t(Theta_hs_out_d_t, Theta_hs_in_d_t, V_hs_supply_d_t, C_df_H_d_t, region)

    # (37)
    E_E_fan_H_d_t = get_E_E_fan_H_d_t(P_fan_rtd_H, V_hs_vent_d_t, V_hs_supply_d_t, V_hs_dsgn_H, q_hs_H_d_t)

    # (20)
    e_th_mid_H = calc_e_th_mid_H(V_fan_mid_H, q_hs_mid_H)

    # (19)
    e_th_rtd_H = calc_e_th_rtd_H(V_fan_rtd_H, q_hs_rtd_H)

    # (17)
    e_th_H_d_t = calc_e_th_H_d_t(Theta_ex_d_t, Theta_hs_in_d_t, Theta_hs_out_d_t, V_hs_supply_d_t)

    # (11)
    e_r_rtd_H = get_e_r_rtd_H(e_th_rtd_H, q_hs_rtd_H, P_hs_rtd_H, P_fan_rtd_H)

    # (15)
    e_r_min_H = get_e_r_min_H(e_r_rtd_H)

    # (13)
    e_r_mid_H = get_e_r_mid_H(e_r_rtd_H, e_th_mid_H, q_hs_mid_H, P_hs_mid_H, P_fan_mid_H, EquipmentSpec)

    # (9)
    e_r_H_d_t = get_e_r_H_d_t(q_hs_H_d_t, q_hs_rtd_H, q_hs_min_H, q_hs_mid_H, e_r_mid_H, e_r_min_H, e_r_rtd_H)

    # (7)
    e_hs_H_d_t = get_e_hs_H_d_t(e_th_H_d_t, e_r_H_d_t)

    # (5)
    E_E_comp_H_d_t = get_E_E_comp_H_d_t(q_hs_H_d_t, e_hs_H_d_t)

    # (1)
    E_E_H_d_t = E_E_comp_H_d_t + E_E_fan_H_d_t

    return E_E_H_d_t


# 日付dの時刻tにおける1時間当たりの冷房時の消費電力量（kWh/h）(2)
def get_E_E_C_d_t(Theta_hs_out_d_t, Theta_hs_in_d_t,  X_hs_out_d_t, X_hs_in_d_t, V_hs_supply_d_t, V_hs_vent_d_t,
                  q_hs_rtd_C, V_hs_dsgn_C, q_hs_mid_C, q_hs_min_C,
                  P_fan_rtd_C, P_fan_mid_C, P_hs_rtd_C, P_hs_mid_C, V_fan_rtd_C, V_fan_mid_C, EquipmentSpec, region):

    # 外気条件
    outdoor = load_outdoor()
    Theta_ex_d_t = get_Theta_ex(region, outdoor)

    # (4)
    q_hs_C_d_t = get_q_hs_C_d_t(Theta_hs_out_d_t, Theta_hs_in_d_t, X_hs_out_d_t, X_hs_in_d_t, V_hs_supply_d_t, region)

    # (38)
    E_E_fan_C_d_t = get_E_E_fan_C_d_t(P_fan_rtd_C, V_hs_vent_d_t, V_hs_supply_d_t, V_hs_dsgn_C, q_hs_C_d_t)

    # (22)
    e_th_mid_C = calc_e_th_mid_C(V_fan_mid_C, q_hs_mid_C)

    # (21)
    e_th_rtd_C = calc_e_th_rtd_C(V_fan_rtd_C, q_hs_rtd_C)

    # (18)
    e_th_C_d_t = calc_e_th_C_d_t(Theta_ex_d_t, Theta_hs_in_d_t, X_hs_in_d_t, Theta_hs_out_d_t, V_hs_supply_d_t)

    # (12)
    e_r_rtd_C = get_e_r_rtd_C(e_th_rtd_C, q_hs_rtd_C, P_hs_rtd_C, P_fan_rtd_C)

    # (16)
    e_r_min_C = get_e_r_min_C(e_r_rtd_C)

    # (14)
    e_r_mid_C = get_e_r_mid_C(e_r_rtd_C, e_th_mid_C, q_hs_mid_C, P_hs_mid_C, P_fan_mid_C, EquipmentSpec)

    # (10)
    e_r_C_d_t = get_e_r_C_d_t(q_hs_C_d_t, q_hs_rtd_C, q_hs_min_C, q_hs_mid_C, e_r_mid_C, e_r_min_C, e_r_rtd_C)

    # (8)
    e_hs_C_d_t = get_e_hs_C_d_t(e_th_C_d_t, e_r_C_d_t)

    # (6)
    E_E_comp_C_d_t = get_E_E_comp_C_d_t(q_hs_C_d_t, e_hs_C_d_t)

    # (2)
    E_E_C_d_t = E_E_comp_C_d_t + E_E_fan_C_d_t

    return E_E_C_d_t

### A.3 熱源機の平均暖房能力・平均冷房能力

暖房期：

$$\large
q_{hs,H,d,t}=max\left( c_{p_{air}}\rho_{air}(\theta_{hs,out,d,t}-\theta_{hs,in,d,t}) \frac{V_{hs,supply,d,t}}{3600} \times \frac{1}{C_{df,H,d,t}}, 0 \right)
$$

$$\large
q_{hs,C,d,t}=0
$$

$$\large
q_{hs,CS,d,t}=0
$$

$$\large
q_{hs,CL,d,t}=0
$$

冷房期：

$$\large
q_{hs,H,d,t}=0
$$

$$\large
q_{hs,C,d,t}=q_{hs,CS,d,t}+q_{hs,CL,d,t}
$$

$$\large
q_{hs,CS,d,t}=max\left( c_{p_{air}}\rho_{air}(\theta_{hs,in,d,t}-\theta_{hs,out,d,t}) \frac{V_{hs,supply,d,t}}{3600}, 0 \right)
$$

$$\large
q_{hs,CL,d,t}=\left\{ \begin{array}{ll}
    max\left( L_{wtr}\rho_{air}(X_{hs,in,d,t}-X_{hs,out,d,t}) \frac{V_{hs,supply,d,t}}{3600} \times 10^3, 0 \right) &(q_{hs,CS,d,t} > 0)\\ 
    0 &(q_{hs,CS,d,t} \leq 0)\\ 
    \end{array}
\right.
$$

中間期：

$$\large
q_{hs,H,d,t}=0
$$

$$\large
q_{hs,C,d,t}=0
$$

$$\large
q_{hs,CS,d,t}=0
$$

$$\large
q_{hs,CL,d,t}=0
$$

$C_{df,H,d,t}$：日付$d$の時刻$t$におけるデフロストに関する暖房出力補正係数(-)  
$q_{hs,C,d,t}$：日付$d$の時刻$t$における1 時間当たりの熱源機の平均冷房能力(W)  
$q_{hs,CL,d,t}$：日付$d$の時刻$t$における1 時間当たりの熱源機の平均冷房潜熱能力(W)  
$q_{hs,CS,d,t}$：日付$d$の時刻$t$における1 時間当たりの熱源機の平均冷房顕熱能力(W)  
$q_{hs,H,d,t}$：日付$d$の時刻$t$における1 時間当たりの熱源機の平均暖房能力(W)  
$V_{hs,supply,d,t}$：日付$d$の時刻$t$における熱源機の風量($m^3$/h)  
$X_{hs,in,d,t}$：日付$d$の時刻$t$における熱源機の入口における絶対湿度(kg/kg(DA))  
$X_{hs,out,d,t}$：日付$d$の時刻$t$における熱源機の出口における絶対湿度(kg/kg(DA))  
$\theta_{hs,in,d,t}$：日付$d$の時刻$t$における熱源機の入口における空気温度(℃)  
$\theta_{hs,out,d,t}$：日付$d$の時刻$t$における熱源機の出口における空気温度(℃)  
$c_{p_{air}}$：空気の比熱(J/(kg・K))  
$L_{wtr}$：水の蒸発潜熱(kJ/kg)  
$\rho_{air}$：空気の密度(kg/$m^3$)  


In [82]:
def get_q_hs_H_d_t(Theta_hs_out_d_t, Theta_hs_in_d_t, V_hs_supply_d_t, C_df_H_d_t, region):
    """(3-1)(3-2)(3-3)

    :param Theta_hs_out_d_t:日付dの時刻tにおける熱源機の出口における空気温度（℃）
    :param Theta_hs_in_d_t:日付dの時刻tにおける熱源機の入口における空気温度（℃）
    :param V_hs_supply_d_t:日付dの時刻tにおける熱源機の風量（m3/h）
    :param C_df_H_d_t:日付dの時刻tにおけるデフロストに関する暖房出力補正係数（-）
    :param region:地域区分
    :return:日付dの時刻tにおける1時間当たりの熱源機の平均暖房能力（-）
    """
    H, C, M = get_season_array_d_t(region)
    c_p_air = get_c_p_air()
    rho_air = get_rho_air()

    q_hs_H_d_t = np.zeros(24 * 365)

    # 暖房期 (3-1)
    q_hs_H_d_t[H] = np.clip(c_p_air * rho_air * (Theta_hs_out_d_t[H] - Theta_hs_in_d_t[H]) \
                    * (V_hs_supply_d_t[H] / 3600) * (1 / C_df_H_d_t[H]), 0, None)

    # 冷房期 (3-2)
    q_hs_H_d_t[C] = 0

    # 中間期 (3-3)
    q_hs_H_d_t[M] = 0

    return q_hs_H_d_t


def get_q_hs_C_d_t(Theta_hs_out_d_t, Theta_hs_in_d_t, X_hs_out_d_t, X_hs_in_d_t,V_hs_supply_d_t, region):
    """(4a-1)(4b-1)(4c-1)(4a-2)(4b-2)(4c-2)(4a-3)(4b-3)(4c-3)

    :param Theta_hs_out_d_t:日付dの時刻tにおける熱源機の出口における空気温度（℃）
    :param Theta_hs_in_d_t:日付dの時刻tにおける熱源機の入口における空気温度（℃）
    :param X_hs_out_d_t:日付dの時刻tにおける熱源機の出口における絶対湿度（kg/kg(DA)）
    :param X_hs_in_d_t:日付dの時刻tにおける熱源機の入口における絶対湿度（kg/kg(DA)）
    :param V_hs_supply_d_t:日付dの時刻tにおける熱源機の風量（m3/h）
    :param region:地域区分
    :return:日付dの時刻tにおける1時間当たりの熱源機の平均冷房能力（-）
    """
    H, C, M = get_season_array_d_t(region)
    c_p_air = get_c_p_air()
    rho_air = get_rho_air()
    L_wtr = get_L_wtr()

    # 暖房期および中間期 (4a-1)(4b-1)(4c-1)(4a-3)(4b-3)(4c-3)
    q_hs_C_d_t = np.zeros(24 * 365)
    q_hs_CS_d_t = np.zeros(24 * 365)
    q_hs_CL_d_t = np.zeros(24 * 365)

    # 冷房期 (4a-2)(4b-2)(4c-2)
    q_hs_CS_d_t[C] = np.clip(c_p_air * rho_air * (Theta_hs_in_d_t[C] - Theta_hs_out_d_t[C]) * (V_hs_supply_d_t[C] / 3600), 0, None)

    Cf = np.logical_and(C, q_hs_CS_d_t > 0)

    q_hs_CL_d_t[Cf] = np.clip(L_wtr * rho_air * (X_hs_in_d_t[Cf] - X_hs_out_d_t[Cf]) * (V_hs_supply_d_t[Cf] / 3600) * 10 ** 3, 0, None)

    q_hs_C_d_t = q_hs_CS_d_t + q_hs_CL_d_t

    return q_hs_C_d_t

### A.4 圧縮機

#### A.4.1 消費電力量

$$\large
E_{E,comp,H,d,t}=\left\{ \begin{array}{ll}
    \frac{q_{hs,H,d,t}}{e_{hs,H,d,t}} \times 10^{-3} &(q_{hs,H,d,t} > 0)\\ 
    0 &(q_{hs,H,d,t} \leq 0)\\ 
    \end{array}
\right.
$$

$$\large
E_{E,comp,C,d,t}=\left\{ \begin{array}{ll}
    \frac{q_{hs,C,d,t}}{e_{hs,C,d,t}} \times 10^{-3} &(q_{hs,C,d,t} > 0)\\ 
    0 &(q_{hs,C,d,t} \leq 0)\\ 
    \end{array}
\right.
$$

$E_{E,comp,C,d,t}$：日付$d$の時刻$t$における 1 時間当たりの冷房時の圧縮機の消費電力量(kWh/h)  
$E_{E,comp,H,d,t}$：日付$d$の時刻$t$における 1 時間当たりの暖房時の圧縮機の消費電力量(kWh/h)  
$e_{hs,C,d,t}$：日付$d$の時刻$t$における冷房時の熱源機の効率(-)  
$e_{hs,H,d,t}$：日付$d$の時刻$t$における暖房時の熱源機の効率(-)  
$q_{hs,C,d,t}$：日付$d$の時刻$t$における1 時間当たりの熱源機の平均冷房能力(W)  
$q_{hs,H,d,t}$：日付$d$の時刻$t$における1 時間当たりの熱源機の平均暖房能力(W)  


In [83]:
def get_E_E_comp_H_d_t(q_hs_H_d_t, e_hs_H_d_t):
    """(5)

    :param q_hs_H_d_t:日付dの時刻tにおける1時間当たりの熱源機の平均暖房能力（-）
    :param e_hs_H_d_t:日付dの時刻tにおける暖房時の熱源機の効率（-）
    :return:日付dの時刻tにおける1時間当たりの暖房時の圧縮機の消費電力量（kWh/h）
    """
    E_E_comp_H_d_t = np.zeros(24 * 365)

    f = q_hs_H_d_t > 0

    E_E_comp_H_d_t[f] = (q_hs_H_d_t[f] / e_hs_H_d_t[f]) * 10 ** (-3)

    return E_E_comp_H_d_t


def get_E_E_comp_C_d_t(q_hs_C_d_t, e_hs_C_d_t):
    """(6)

    :param q_hs_C_d_t:日付dの時刻tにおける1時間当たりの熱源機の平均冷房能力（-）
    :param e_hs_C_d_t:日付dの時刻tにおける冷房時の熱源機の効率（-）
    :return:日付dの時刻tにおける1時間当たりの冷房時の圧縮機の消費電力量（kWh/h）
    """
    E_E_comp_C_d_t = np.zeros(24 * 365)

    f = q_hs_C_d_t > 0

    E_E_comp_C_d_t[f] = (q_hs_C_d_t[f] / e_hs_C_d_t[f]) * 10 ** (-3)

    return E_E_comp_C_d_t

$$\large
e_{hs,H,d,t}=e_{th,H,d,t}\times e_{r,H,d,t}
$$

$$\large
e_{hs,C,d,t}=e_{th,C,d,t}\times e_{r,C,d,t}
$$

$e_{hs,C,d,t}$：日付$d$の時刻$t$における冷房時の熱源機の効率(-)  
$e_{hs,H,d,t}$：日付$d$の時刻$t$における暖房時の熱源機の効率(-)  
$e_{r,C,d,t}$：日付$d$の時刻$t$における冷房時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,H,d,t}$：日付$d$の時刻$t$における暖房時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{th,C,d,t}$：日付$d$の時刻$t$における冷房時のヒートポンプサイクルの理論効率(-)  
$e_{th,H,d,t}$：日付$d$の時刻$t$における暖房時のヒートポンプサイクルの理論効率(-)  


In [84]:
def get_e_hs_H_d_t(e_th_H_d_t, e_r_H_d_t):
    """(7)

    :param e_th_H_d_t:日付dの時刻tにおける暖房時のヒートポンプサイクルの理論効率（-）
    :param e_r_H_d_t:日付dの時刻tにおける暖房時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    :return:日付dの時刻tにおける暖房時の熱源機の効率（-）
    """
    return e_th_H_d_t * e_r_H_d_t


def get_e_hs_C_d_t(e_th_C_d_t, e_r_C_d_t):
    """(8)

    :param e_th_C_d_t:日付dの時刻tにおける冷房時のヒートポンプサイクルの理論効率（-）
    :param e_r_C_d_t:日付dの時刻tにおける冷房時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    :return:日付dの時刻tにおける冷房時の熱源機の効率（-）
    """
    return e_th_C_d_t * e_r_C_d_t

#### A.4.2 ヒートポンプサイクルの理論効率に対する熱源機の効率の比

#### A.4.2.1 エネルギー消費量の算定におけるヒートポンプサイクルの理論効率に対する熱源機の効率の比

暖房時：

$q_{hs,H,d,t} \leq q_{hs,min,H}$の場合：

$$\large
e_{r,H,d,t}=e_{r,min,H} - (q_{hs,min,H}-q_{hs,H,d,t}) \times \frac{e_{r,min,H}}{q_{hs,min,H}}
$$

$q_{hs,min,H} < q_{hs,H,d,t} \leq q_{hs,mid,H}$の場合：

$$\large
e_{r,H,d,t}=e_{r,mid,H} - (q_{hs,mid,H}-q_{hs,H,d,t}) \times \frac{e_{r,mid,H}-e_{r,min,H}}{q_{hs,mid,H}-q_{hs,min,H}}
$$

$q_{hs,mid,H} < q_{hs,H,d,t} \leq q_{hs,rtd,H}$の場合：

$$\large
e_{r,H,d,t}=e_{r,rtd,H} - (q_{hs,rtd,H}-q_{hs,H,d,t}) \times \frac{e_{r,rtd,H}-e_{r,mid,H}}{q_{hs,rtd,H}-q_{hs,mid,H}}
$$

$q_{hs,rtd,H} < q_{hs,H,d,t}$の場合：

$$\large
e_{r,H,d,t}=\left\{ \begin{array}{ll}
    max\left(e_{r,rtd,H}-(q_{hs,H,d,t}-q_{hs,rtd,H})\times \frac{e_{r,rtd,H}}{q_{hs,rtd,H}},0.4\right) &(e_{r,rtd,H} > 0.4)\\ 
    e_{r,rtd,H} &(e_{r,rtd,H} \leq 0.4)\\ 
    \end{array}
\right.
$$

冷房時：

$q_{hs,C,d,t} \leq q_{hs,min,C}$の場合：

$$\large
e_{r,C,d,t}=e_{r,min,C} - (q_{hs,min,C}-q_{hs,C,d,t}) \times \frac{e_{r,min,C}}{q_{hs,min,C}}
$$

$q_{hs,min,C} < q_{hs,C,d,t} \leq q_{hs,mid,C}$の場合：

$$\large
e_{r,C,d,t}=e_{r,mid,C} - (q_{hs,mid,C}-q_{hs,C,d,t}) \times \frac{e_{r,mid,C}-e_{r,min,C}}{q_{hs,mid,C}-q_{hs,min,C}}
$$

$q_{hs,mid,C} < q_{hs,C,d,t} \leq q_{hs,rtd,C}$の場合：

$$\large
e_{r,C,d,t}=e_{r,rtd,C} - (q_{hs,rtd,C}-q_{hs,C,d,t}) \times \frac{e_{r,rtd,C}-e_{r,mid,C}}{q_{hs,rtd,C}-q_{hs,mid,C}}
$$

$q_{hs,rtd,C} < q_{hs,C,d,t}$の場合：

$$\large
e_{r,C,d,t}=\left\{ \begin{array}{ll}
    max\left(e_{r,rtd,C}-(q_{hs,C,d,t}-q_{hs,rtd,C})\times \frac{e_{r,rtd,C}}{q_{hs,rtd,C}},0.4\right) &(e_{r,rtd,C} > 0.4)\\ 
    e_{r,rtd,C} &(e_{r,rtd,C} \leq 0.4)\\ 
    \end{array}
\right.
$$

$q_{hs,C,d,t}$：日付$d$の時刻$t$における 1 時間当たりの熱源機の平均冷房能力(W)  
$q_{hs,mid,C}$：熱源機の中間冷房能力(W)  
$q_{hs,min,C}$：熱源機の最小冷房能力(W)  
$q_{hs,rtd,C}$：熱源機の定格冷房能力(W)  
$q_{hs,H,d,t}$：日付$d$の時刻$t$における 1 時間当たりの熱源機の平均暖房能力(W)  
$q_{hs,mid,H}$：熱源機の中間暖房能力(W)  
$q_{hs,min,H}$：熱源機の最小暖房能力(W)  
$q_{hs,rtd,H}$：熱源機の定格暖房能力(W)  
$e_{r,C,d,t}$：日付$d$の時刻$t$における冷房時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,mid,C}$：中間冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,min,C}$：最小冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,rtd,C}$：定格冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,H,d,t}$：日付$d$の時刻$t$における暖房時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,mid,H}$：中間暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,min,H}$：最小暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,rtd,H}$：定格暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  


In [85]:
def get_e_r_H_d_t(q_hs_H_d_t, q_hs_rtd_H, q_hs_min_H, q_hs_mid_H, e_r_mid_H, e_r_min_H, e_r_rtd_H):
    """(9-1)(9-2)(9-3)(9-4)

    :param q_hs_H_d_t: 日付dの時刻tにおける1時間当たりの熱源機の平均暖房能力（-）
    :param q_hs_rtd_H: 熱源機の定格暖房能力（-）
    :param q_hs_min_H: 熱源機の最小暖房能力（-）
    :param q_hs_mid_H: 熱源機の中間暖房能力（-）
    :param e_r_mid_H: 中間暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    :param e_r_min_H: 最小暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    :param e_r_rtd_H: 定格暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    :return: 日付dの時刻tにおける暖房時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    """
    f1 = q_hs_H_d_t <= q_hs_min_H
    f2 = (q_hs_min_H < q_hs_H_d_t) & (q_hs_H_d_t <= q_hs_mid_H)
    f3 = (q_hs_mid_H < q_hs_H_d_t) & (q_hs_H_d_t <= q_hs_rtd_H)
    f4 = q_hs_rtd_H < q_hs_H_d_t

    e_r_H_d_t = np.zeros(24 * 365)

    # (9-1)
    e_r_H_d_t[f1] = e_r_min_H - (q_hs_min_H - q_hs_H_d_t[f1]) * (e_r_min_H / q_hs_min_H)

    # (9-2)
    e_r_H_d_t[f2] = e_r_mid_H - (q_hs_mid_H - q_hs_H_d_t[f2]) \
                    * ((e_r_mid_H - e_r_min_H) / (q_hs_mid_H - q_hs_min_H))

    # (9-3)
    e_r_H_d_t[f3] = e_r_rtd_H - (q_hs_rtd_H - q_hs_H_d_t[f3]) \
                    * ((e_r_rtd_H - e_r_mid_H) / (q_hs_rtd_H - q_hs_mid_H))

    # (9-4)
    f5 = np.logical_and(f4, e_r_rtd_H > 0.4)
    f6 = np.logical_and(f4, e_r_rtd_H <= 0.4)

    e_r_H_d_t[f5] = np.clip(e_r_rtd_H - (q_hs_H_d_t[f5] - q_hs_rtd_H) * (e_r_rtd_H / q_hs_rtd_H), 0.4, None)

    e_r_H_d_t[f6] = e_r_rtd_H

    return e_r_H_d_t


def get_e_r_C_d_t(q_hs_C_d_t, q_hs_rtd_C, q_hs_min_C, q_hs_mid_C, e_r_mid_C, e_r_min_C, e_r_rtd_C):
    """(10-1)(10-2)(10-3)(10-4)

    :param q_hs_C_d_t: 日付dの時刻tにおける1時間当たりの熱源機の平均冷房能力（-）
    :param q_hs_rtd_C: 熱源機の定格冷房能力（-）
    :param q_hs_min_C: 熱源機の最小冷房能力（-）
    :param q_hs_mid_C: 熱源機の中間冷房能力（-）
    :param e_r_mid_C: 中間冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    :param e_r_min_C: 最小冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    :param e_r_rtd_C: 定格冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    :return: 日付dの時刻tにおける冷房時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    """
    f1 = q_hs_C_d_t <= q_hs_min_C
    f2 = (q_hs_min_C < q_hs_C_d_t) & (q_hs_C_d_t <= q_hs_mid_C)
    f3 = (q_hs_mid_C < q_hs_C_d_t) & (q_hs_C_d_t <= q_hs_rtd_C)
    f4 = q_hs_rtd_C < q_hs_C_d_t

    e_r_C_d_t = np.zeros(24 * 365)

    # (10-1)
    e_r_C_d_t[f1] = e_r_min_C - (q_hs_min_C - q_hs_C_d_t[f1]) * (e_r_min_C / q_hs_min_C)

    # (10-2)
    e_r_C_d_t[f2] = e_r_mid_C - (q_hs_mid_C - q_hs_C_d_t[f2]) \
                    * ((e_r_mid_C - e_r_min_C) / (q_hs_mid_C - q_hs_min_C))

    # (10-3)
    e_r_C_d_t[f3] = e_r_rtd_C - (q_hs_rtd_C - q_hs_C_d_t[f3]) \
                    * ((e_r_rtd_C - e_r_mid_C) / (q_hs_rtd_C - q_hs_mid_C))

    # (10-4)
    f5 = np.logical_and(f4, e_r_rtd_C > 0.4)
    f6 = np.logical_and(f4, e_r_rtd_C <= 0.4)

    e_r_C_d_t[f5] = np.clip(e_r_rtd_C - (q_hs_C_d_t[f5] - q_hs_rtd_C) * (e_r_rtd_C / q_hs_rtd_C), 0.4, None)

    e_r_C_d_t[f6] = e_r_rtd_C

    return e_r_C_d_t

#### A.4.2.2 JIS 試験におけるヒートポンプサイクルの理論効率に対する熱源機の効率の比

定格暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比$e_{r,rtd,H}$および定格冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比$e_{r,rtd,C}$は、それぞれ、0 を下回る場合は 0 に等しいとし、1.0 を上回る場合は 1.0 に等しいとする。

定格暖房能力運転時：

$$\large
e_{r,rtd,H}=\frac{e_{hs,rtd,H}}{e_{th,rtd,H}}
$$

$$\large
e_{hs,rtd,H}=\frac{q_{hs,rtd,H}}{P_{hs,rtd,H}-P_{fan,rtd,H}}
$$

定格冷房能力運転時：

$$\large
e_{r,rtd,C}=\frac{e_{hs,rtd,C}}{e_{th,rtd,C}}
$$

$$\large
e_{hs,rtd,C}=\frac{q_{hs,rtd,C}}{P_{hs,rtd,C}-P_{fan,rtd,C}}
$$

$e_{hs,rtd,C}$：定格冷房能力運転時の熱源機の効率(-)  
$e_{hs,rtd,H}$：定格暖房能力運転時の熱源機の効率(-)  
$e_{th,rtd,C}$：定格冷房能力運転時のヒートポンプサイクルの理論効率(-)  
$e_{th,rtd,H}$：定格暖房能力運転時のヒートポンプサイクルの理論効率(-)  
$P_{hs,rtd,C}$：熱源機の定格冷房消費電力(W)  
$P_{hs,rtd,H}$：熱源機の定格暖房消費電力(W)  
$P_{fan,rtd,C}$：定格冷房能力運転時の送風機の消費電力(W)  
$P_{fan,rtd,H}$：定格暖房能力運転時の送風機の消費電力(W)  
$q_{hs,rtd,C}$：熱源機の定格冷房能力(W)  
$q_{hs,rtd,H}$：熱源機の定格暖房能力(W)  
$e_{r,rtd,C}$：定格冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,rtd,H}$：定格暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  


In [86]:
def get_e_r_rtd_H(e_th_rtd_H, q_hs_rtd_H, P_hs_rtd_H, P_fan_rtd_H):
    """(11a)(11b)

    :param e_th_rtd_H: 定格暖房能力運転時のヒートポンプサイクルの理論効率（-）
    :param q_hs_rtd_H: 熱源機の定格暖房能力（-）
    :param P_hs_rtd_H: 熱源機の定格暖房消費電力（W）
    :param P_fan_rtd_H: 定格暖房能力運転時の送風機の消費電力（W）
    :return: 定格暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    """
    # 定格暖房能力運転時の熱源機の効率（-）(11b)
    if P_hs_rtd_H <= P_fan_rtd_H:
        raise ValueError(P_fan_rtd_H)

    e_hs_rtd_H = q_hs_rtd_H / (P_hs_rtd_H - P_fan_rtd_H)

    # (11a)
    e_r_rtd_H = e_hs_rtd_H / e_th_rtd_H

    e_r_rtd_H = np.clip(e_r_rtd_H, 0, 1.0)

    return e_r_rtd_H


def get_e_r_rtd_C(e_th_rtd_C, q_hs_rtd_C, P_hs_rtd_C, P_fan_rtd_C):
    """(12a)(12b)

    :param e_th_rtd_C: 定格冷房能力運転時のヒートポンプサイクルの理論効率（-）
    :param q_hs_rtd_C: 熱源機の定格冷房能力（-）
    :param P_hs_rtd_C: 熱源機の定格冷房消費電力（W）
    :param P_fan_rtd_C: 定格冷房能力運転時の送風機の消費電力（W）
    :return: 定格冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    """
    # 定格暖房能力運転時の熱源機の効率（-）(12b)
    if P_hs_rtd_C <= P_fan_rtd_C:
        raise ValueError(P_hs_rtd_C)

    e_hs_rtd_C = q_hs_rtd_C / (P_hs_rtd_C - P_fan_rtd_C)

    # (12a)
    e_r_rtd_C = e_hs_rtd_C / e_th_rtd_C

    e_r_rtd_C = np.clip(e_r_rtd_C, 0, 1.0)

    return e_r_rtd_C

中間暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比$e_{r,mid,H}$および中間冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比$e_{r,mid,C}$は、それぞれ、0 を下回る場合は 0 に等しいとし、1.0 を上回る場合は 1.0 に等しいとする。

中間暖房能力運転時：

性能試験により得られる値を用いる場合：

$$\large
e_{r,mid,H}=\frac{e_{hs,mid,H}}{e_{th,mid,H}}
$$

$$\large
e_{hs,mid,H}=\frac{q_{hs,mid,H}}{P_{hs,mid,H}-P_{fan,mid,H}}
$$

性能試験により得られる値を用いない場合：

$$\large
e_{r,mid,H}=e_{r,rtd,H} \times 0.95
$$

中間冷房能力運転時：

性能試験により得られる値を用いる場合：

$$\large
e_{r,mid,C}=\frac{e_{hs,mid,C}}{e_{th,mid,C}}
$$

$$\large
e_{hs,mid,C}=\frac{q_{hs,mid,C}}{P_{hs,mid,C}-P_{fan,mid,C}}
$$

性能試験により得られる値を用いない場合：

$$\large
e_{r,mid,C}=e_{r,rtd,C} \times 0.95
$$

$e_{hs,mid,C}$：中間冷房能力運転時の熱源機の効率(-)  
$e_{hs,mid,H}$：中間暖房能力運転時の熱源機の効率(-)  
$e_{th,mid,C}$：中間冷房能力運転時のヒートポンプサイクルの理論効率(-)  
$e_{th,mid,H}$：中間暖房能力運転時のヒートポンプサイクルの理論効率(-)  
$P_{hs,mid,C}$：熱源機の中間冷房消費電力(W)  
$P_{hs,mid,H}$：熱源機の中間暖房消費電力(W)  
$P_{fan,mid,C}$：中間冷房能力運転時の送風機の消費電力(W)  
$P_{fan,mid,H}$：中間暖房能力運転時の送風機の消費電力(W)  
$q_{hs,mid,C}$：中間冷房能力(W)  
$q_{hs,mid,H}$：中間暖房能力(W)  
$e_{r,mid,C}$：中間冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,mid,H}$：中間暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,rtd,C}$：定格冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,rtd,H}$：定格暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  


In [87]:
def get_e_r_mid_H(e_r_rtd_H, e_th_mid_H, q_hs_mid_H, P_hs_mid_H, P_fan_mid_H, EquipmentSpec):
    """(13a-1)(13b)(13a-2)

    :param e_r_rtd_H:定格暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    :param e_th_mid_H:中間暖房能力運転時のヒートポンプサイクルの理論効率（-）
    :param q_hs_mid_H:中間暖房能力（-）※ 付録B
    :param P_hs_mid_H:熱源機の中間暖房消費電力（W）※ 付録B
    :param P_fan_mid_H:中間暖房能力運転時の送風機の消費電力（W）※ 付録B
    :param EquipmentSpec:機器の仕様
    :return:中間暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    """
    # (13a-2)
    if EquipmentSpec == '入力しない' or EquipmentSpec == '定格能力試験の値を入力する':

        e_r_mid_H = e_r_rtd_H * 0.95

    # (13a-1)(13b)
    elif EquipmentSpec == '定格能力試験と中間能力試験の値を入力する':

        # 中間暖房能力運転時の熱源機の効率（-）
        if P_hs_mid_H <= P_fan_mid_H:
            raise ValueError(P_fan_mid_H)

        e_hs_mid_H = q_hs_mid_H / (P_hs_mid_H - P_fan_mid_H)

        e_r_mid_H = e_hs_mid_H / e_th_mid_H

    else:
        raise ValueError(EquipmentSpec)

    e_r_mid_H = np.clip(e_r_mid_H, 0, 1.0)

    return e_r_mid_H


def get_e_r_mid_C(e_r_rtd_C, e_th_mid_C, q_hs_mid_C, P_hs_mid_C, P_fan_mid_C, EquipmentSpec):
    """(14a-1)(14b)(14a-2)

    :param e_r_rtd_C:定格冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    :param e_th_mid_C:中間冷房能力運転時のヒートポンプサイクルの理論効率（-）
    :param q_hs_mid_C:中間冷房能力（-）※付録B
    :param P_hs_mid_C:熱源機の中間冷房消費電力（W）※付録B
    :param P_fan_mid_C:中間冷房能力運転時の送風機の消費電力（W）※付録B
    :param EquipmentSpec:機器の仕様
    :return:中間冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    """
    # (14a-2)
    if EquipmentSpec == '入力しない' or EquipmentSpec == '定格能力試験の値を入力する':

        e_r_mid_C = e_r_rtd_C * 0.95

    # (14a-1)(14b)
    elif EquipmentSpec == '定格能力試験と中間能力試験の値を入力する':

        # 中間暖房能力運転時の熱源機の効率（-）
        if P_hs_mid_C <= P_fan_mid_C:
            raise ValueError(P_hs_mid_C)

        e_hs_mid_C = q_hs_mid_C / (P_hs_mid_C - P_fan_mid_C)

        e_r_mid_C = e_hs_mid_C / e_th_mid_C

    else:
        raise ValueError(EquipmentSpec)

    e_r_mid_C = np.clip(e_r_mid_C, 0, 1.0)

    return e_r_mid_C

最小暖房能力運転時：

$$\large
e_{r,min,H}=e_{r,rtd,H} \times 0.65
$$

最小冷房能力運転時：

$$\large
e_{r,min,C}=e_{r,rtd,C} \times 0.65
$$

$e_{r,min,C}$：最小冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,min,H}$：最小暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,rtd,C}$：定格冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  
$e_{r,rtd,H}$：定格暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比(-)  


In [88]:
def get_e_r_min_H(e_r_rtd_H):
    """(15)

    :param e_r_rtd_H:定格暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    :return:最小暖房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    """
    return e_r_rtd_H * 0.65


def get_e_r_min_C(e_r_rtd_C):
    """(16)

    :param e_r_rtd_C:定格冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    :return:最小冷房能力運転時のヒートポンプサイクルの理論効率に対する熱源機の効率の比（-）
    """
    return e_r_rtd_C * 0.65

#### A.4.3 ヒートポンプサイクルの理論効率

#### A.4.3.1 エネルギー消費量の算定におけるヒートポンプサイクルの理論効率

$$\large
e_{th,H,d,t}=e'_{th,d,t}
$$

$$\large
e_{th,C,d,t}=e'_{th,d,t}-1
$$

$e_{th,C,d,t}$：日付$d$の時刻$t$における冷房時のヒートポンプサイクルの理論効率(-)  
$e_{th,H,d,t}$：日付$d$の時刻$t$における暖房時のヒートポンプサイクルの理論効率(-)  
$e'_{th,d,t}$：日付$d$の時刻$t$におけるヒートポンプサイクル（加熱）の理論効率(-)  


In [89]:
def calc_e_th_H_d_t(Theta_ex_d_t, Theta_hs_in_d_t, Theta_hs_out_d_t, V_hs_supply_d_t):
    """(17)

    :param Theta_ex_d_t: 日付dの時刻tにおける外気温
    :param Theta_hs_in_d_t: 日付dの時刻tにおける熱源機の入口における空気温度
    :param Theta_hs_out_d_t: 日付dの時刻tにおける熱源機の出口における空気温度
    :param V_hs_supply_d_t: 日付dの時刻tにおける熱源機の風量
    :param alpha_c_hex_H: 暖房時の室内熱交換器表面の顕熱伝達率（W/(m2・K)）
    :return: 日付dの時刻tにおける暖房時のヒートポンプサイクルの理論効率（-）
    """
    e_dash_th_d_t = np.zeros(24 * 365)
    for i in range(24 * 365):
        # (35)
        alpha_c_hex_H = get_alpha_c_hex_H(V_hs_supply_d_t[i])

        # (31)
        Theta_sur_f_hex_H = get_Theta_sur_f_hex_H_calc(Theta_hs_in_d_t[i], Theta_hs_out_d_t[i], V_hs_supply_d_t[i],
                                                       alpha_c_hex_H)

        # (23)
        Theta_ref_cnd_H = get_Theta_ref_cnd_H(Theta_sur_f_hex_H)

        # (24)
        Theta_ref_evp_H = get_Theta_ref_evp_H(Theta_ex_d_t[i], Theta_ref_cnd_H)

        # (25)
        Theta_ref_SC_H = get_Theta_ref_SC_H(Theta_ref_cnd_H)

        # (26)
        Theta_ref_SH_H = get_Theta_ref_SH_H(Theta_ref_cnd_H)

        # 4_8_a (1) ヒートポンプサイクルの理論暖房効率
        e_dash_th_d_t[i] = calc_e_ref_H_th(Theta_ref_evp_H, Theta_ref_cnd_H, Theta_ref_SC_H, Theta_ref_SH_H)

    # (17)
    return e_dash_th_d_t


def calc_e_th_C_d_t(Theta_ex_d_t, Theta_hs_in_d_t, X_hs_in_d_t, Theta_hs_out_d_t, V_hs_supply_d_t):
    """(18)

    :param Theta_ex_d_t: 日付dの時刻tにおける外気温
    :param Theta_hs_in_d_t: 日付dの時刻tにおける熱源機の入口における空気温度
    :param X_hs_in_d_t: 日付dの時刻tにおける熱源機の入口における絶対湿度
    :param Theta_hs_out_d_t: 日付dの時刻tにおける熱源機の出口における空気温度
    :param V_hs_supply_d_t: 日付dの時刻tにおける熱源機の風量
    :param alpha_c_hex_C: 冷房時の室内熱交換器表面の顕熱伝達率（W/(m2・K)）
    :return: 日付dの時刻tにおける暖房時のヒートポンプサイクルの理論効率（-）
    """
    e_dash_th_d_t = np.zeros(24 * 365)
    Theta_sur_f_hex_C = np.zeros(24 * 365)
    for i in range(24 * 365):
        # (36)
        alpha_c_hex_C, alpha_dash_c_hex_C = get_alpha_c_hex_C(V_hs_supply_d_t[i], X_hs_in_d_t[i])

        # (32)
        Theta_sur_f_hex_C[i] = get_Theta_sur_f_hex_C_calc(Theta_hs_in_d_t[i], Theta_hs_out_d_t[i], V_hs_supply_d_t[i],
                                                       alpha_c_hex_C)

        # (28)
        Theta_ref_evp_C = get_Theta_ref_evp_C(Theta_sur_f_hex_C[i])

        # (27)
        Theta_ref_cnd_C = get_Theta_ref_cnd_C(Theta_ex_d_t[i], Theta_ref_evp_C)

        # (29)
        Theta_ref_SC_C = get_Theta_ref_SC_C(Theta_ref_cnd_C)

        # (30)
        Theta_ref_SH_C = get_Theta_ref_SH_C(Theta_ref_cnd_C)

        # 4_8_a (1) ヒートポンプサイクルの理論暖房効率
        e_dash_th_d_t[i] = calc_e_ref_H_th(Theta_ref_evp_C, Theta_ref_cnd_C, Theta_ref_SC_C, Theta_ref_SH_C)

    # (18)
    return e_dash_th_d_t - 1

<div style="text-align: center;font-weight: bold;">表　算定式における記号に対応する変数</div>

<table>
<thead>
<tr>
<th colspan="2" rowspan="2" style="text-align: center;">算定式における記号</th>
<th colspan="2" style="text-align: center;">算定式における記号に対応する変数</th>
</tr>
<tr>
<th>暖房時のヒートポンプサイクル<br>
の理論効率$e_{th,H,d,t}$（式 17）を<br>
算定する場合</th>
<th>冷房時のヒートポンプサイクル<br>
の理論効率$e_{th,C,d,t}$（式 18）を<br>
算定する場合</th>
</tr>
</thead>
<tbody>
<tr>
    <th>暖房時の蒸発温度<br>$\theta_{ref,env,H}$（式 24）<br>または<br>冷房時の凝縮温度<br>$\theta_{ref,cnd,C}$（式 27）</th>
    <th>外気温度(℃)$\theta_{ex}$</th>
    <td>日付$d$の時刻$t$における外気温度$\theta_{ex,d,t}$</td>
    <td>日付$d$の時刻$t$における外気温度$\theta_{ex,d,t}$</td>
</tr>
<tr>
    <th rowspan="4">暖房時の室内機熱交換器の<br>表面温度$\theta_{surf,hex,H}$（式 31）<br>または<br>冷房時の室内機熱交換器の<br>表面温度$\theta_{surf,hex,C}$（式 32）</th>
    <th>熱源機の入口における空気温度(℃)$\theta_{hs,in}$</th>
    <td>日付$d$の時刻$t$における熱源機の入口における空気温度$\theta_{hs,in,d,t}$</td>
    <td>日付$d$の時刻$t$における熱源機の入口における空気温度$\theta_{hs,in,d,t}$</td>
</tr>
<tr>
    <th>熱源機の入口における絶対湿度(kg/kg(DA))$X_{hs,in}$</th>
    <td>-</td>
    <td>日付$d$の時刻$t$における熱源機の入口における絶対湿度$X_{hs,in,d,t}$</td>
</tr>
<tr>
    <th>熱源機の出口における空気温度(℃)$\theta_{hs,out}$</th>
    <td>日付$d$の時刻$t$における熱源機の出口における空気温度$\theta_{hs,out,d,t}$</td>
    <td>日付$d$の時刻$t$における熱源機の出口における空気温度$\theta_{hs,out,d,t}$</td>
</tr>
<tr>
    <th>熱源機の風量($m^3$/h)$V_{hs,supply}$</th>
    <td>日付$d$の時刻$t$における熱源機の風量$V_{hs,supply,d,t}$</td>
    <td>日付$d$の時刻$t$における熱源機の風量$V_{hs,supply,d,t}$</td>
</tr>
</tbody>
</table>

#### A.4.3.2 JIS 試験におけるヒートポンプサイクルの理論効率

$$\large
e_{th,rtd,H}=e'_{th,rtd,H}
$$

$$\large
e_{th,mid,H}=e'_{th,mid,H}
$$

$$\large
e_{th,rtd,C}=e'_{th,rtd,C}-1
$$

$$\large
e_{th,mid,C}=e'_{th,mid,C}-1
$$

$e_{th,mid,C}$：中間冷房能力運転時のヒートポンプサイクルの理論効率(-)  
$e_{th,mid,H}$：中間暖房能力運転時のヒートポンプサイクルの理論効率(-)  
$e_{th,rtd,C}$：定格冷房能力運転時のヒートポンプサイクルの理論効率(-)  
$e_{th,rtd,H}$：定格暖房能力運転時のヒートポンプサイクルの理論効率(-)  
$e'_{th,mid,C}$：中間冷房能力運転時のヒートポンプサイクル（加熱）の理論効率(-)  
$e'_{th,mid,H}$：中間暖房能力運転時のヒートポンプサイクル（加熱）の理論効率(-)  
$e'_{th,rtd,C}$：定格冷房能力運転時のヒートポンプサイクル（加熱）の理論効率(-)  
$e'_{th,rtd,H}$：定格暖房能力運転時のヒートポンプサイクル（加熱）の理論効率(-)  


In [90]:
def calc_e_th_rtd_H(V_fan_rtd_H, q_hs_rtd_H):
    """(19)

    :param V_fan_rtd_H: 定格暖房能力運転時の送風機の風量（m3/h）
    :param q_hs_rtd_H: 定格暖房能力（W）
    :param alpha_c_hex_H: 暖房時の室内熱交換器表面の顕熱伝達率（W/(m2・K)）
    :return: 定格暖房能力運転時のヒートポンプサイクルサイクルの理論効率（-）
    """
    # (35)
    alpha_c_hex_H = get_alpha_c_hex_H(V_fan_rtd_H)

    # (33)
    Theta_sur_f_hex_H = get_Theta_sur_f_hex_H_JIS(V_fan_rtd_H, q_hs_rtd_H, alpha_c_hex_H)

    # (23)
    Theta_ref_cnd_H = get_Theta_ref_cnd_H(Theta_sur_f_hex_H)

    # (24)
    Theta_ref_evp_H = get_Theta_ref_evp_H(7, Theta_ref_cnd_H)

     # (25)
    Theta_ref_SC_H = get_Theta_ref_SC_H(Theta_ref_cnd_H)

    # (26)
    Theta_ref_SH_H = get_Theta_ref_SH_H(Theta_ref_cnd_H)

    # 4_8_a (1) ヒートポンプサイクルの理論暖房効率
    e_dash_th_rtd_H = calc_e_ref_H_th(Theta_ref_evp_H, Theta_ref_cnd_H, Theta_ref_SC_H, Theta_ref_SH_H)

    # (19)
    return e_dash_th_rtd_H


def calc_e_th_mid_H(V_fan_mid_H, q_hs_mid_H):
    """(20)

    :param V_fan_mid_H: 中間暖房能力運転時の送風機の風量（m3/h）
    :param q_hs_mid_H: 熱源機の中間暖房能力（W）
    :param alpha_c_hex_H: 暖房時の室内熱交換器表面の顕熱伝達率（W/(m2・K)）
    :return: 中間暖房能力運転時のヒートポンプサイクルサイクルの理論効率（-）
    """
    # (35)
    alpha_c_hex_H = get_alpha_c_hex_H(V_fan_mid_H)

    # (33)
    Theta_sur_f_hex_H = get_Theta_sur_f_hex_H_JIS(V_fan_mid_H, q_hs_mid_H, alpha_c_hex_H)

    # (23)
    Theta_ref_cnd_H = get_Theta_ref_cnd_H(Theta_sur_f_hex_H)

    # (24)
    Theta_ref_evp_H = get_Theta_ref_evp_H(7, Theta_ref_cnd_H)

     # (25)
    Theta_ref_SC_H = get_Theta_ref_SC_H(Theta_ref_cnd_H)

    # (26)
    Theta_ref_SH_H = get_Theta_ref_SH_H(Theta_ref_cnd_H)

    # 4_8_a (1) ヒートポンプサイクルの理論暖房効率
    e_dash_th_mid_H = calc_e_ref_H_th(Theta_ref_evp_H, Theta_ref_cnd_H, Theta_ref_SC_H, Theta_ref_SH_H)

    # (20)
    return e_dash_th_mid_H


def calc_e_th_rtd_C(V_fan_rtd_C, q_hs_rtd_C):
    """(21)

    :param V_fan_rtd_H: 定格冷房能力運転時の送風機の風量（m3/h）
    :param q_hs_rtd_H: 定格冷房能力（W）
    :return: 定格冷房能力運転時のヒートポンプサイクルサイクルの理論効率（-）
    """
    # 表5より
    X_hs_in = 0.010376
    # (36)
    alpha_c_hex_C, alpha_dash_c_hex_C = get_alpha_c_hex_C(V_fan_rtd_C, X_hs_in)

    # (34)
    def func(x):
        # 表5より
        q_hs_C = q_hs_rtd_C

        # 連立方程式を解くために(34a)の左辺を移項し、左辺を0にしておく
        return calc_Theta_sur_f_hex_C_JIS(x, V_fan_rtd_C, alpha_c_hex_C, alpha_dash_c_hex_C) - q_hs_C

    # x = fsolve(fun,x0) は、点 x0 を開始点として方程式 fun(x) = 0 (ゼロの配列) の解を求めようとする
    Theta_sur_f_hex_C = optimize.bisect(func, -273.15, 99.96)

    # (28)
    Theta_ref_evp_C = get_Theta_ref_evp_C(Theta_sur_f_hex_C)

    # (27)
    Theta_ref_cnd_C = get_Theta_ref_cnd_C(35, Theta_ref_evp_C)

    # (29)
    Theta_ref_SC_C = get_Theta_ref_SC_C(Theta_ref_cnd_C)

    # (30)
    Theta_ref_SH_C = get_Theta_ref_SH_C(Theta_ref_cnd_C)

    # 4_8_a (1) ヒートポンプサイクルの理論暖房効率
    e_dash_th_rtd_C = calc_e_ref_H_th(Theta_ref_evp_C, Theta_ref_cnd_C, Theta_ref_SC_C, Theta_ref_SH_C)

    # (21)
    return e_dash_th_rtd_C - 1


def calc_e_th_mid_C(V_fan_mid_C, q_hs_mid_C):
    """(22)

    :param V_fan_rtd_H: 定格冷房能力運転時の送風機の風量（m3/h）
    :param q_hs_rtd_H: 定格冷房能力（W）
    :return: 定格冷房能力運転時のヒートポンプサイクルサイクルの理論効率（-）
    """
    # 表5より
    X_hs_in = 0.010376
    # (36)
    alpha_c_hex_C, alpha_dash_c_hex_C = get_alpha_c_hex_C(V_fan_mid_C, X_hs_in)

    # (34)
    def func(x):
        # 表5より
        q_hs_C = q_hs_mid_C

        # 連立方程式を解くために(34a)の左辺を移項し、左辺を0にしておく
        return calc_Theta_sur_f_hex_C_JIS(x, V_fan_mid_C, alpha_c_hex_C, alpha_dash_c_hex_C) - q_hs_C

    # x = fsolve(fun,x0) は、点 x0 を開始点として方程式 fun(x) = 0 (ゼロの配列) の解を求めようとする
    Theta_sur_f_hex_C = optimize.bisect(func, -273.15, 99.96)

    # (28)
    Theta_ref_evp_C = get_Theta_ref_evp_C(Theta_sur_f_hex_C)

    # (27)
    Theta_ref_cnd_C = get_Theta_ref_cnd_C(35, Theta_ref_evp_C)

    # (29)
    Theta_ref_SC_C = get_Theta_ref_SC_C(Theta_ref_cnd_C)

    # (30)
    Theta_ref_SH_C = get_Theta_ref_SH_C(Theta_ref_cnd_C)

    # 4_8_a (1) ヒートポンプサイクルの理論暖房効率
    e_dash_th_mid_C = calc_e_ref_H_th(Theta_ref_evp_C, Theta_ref_cnd_C, Theta_ref_SC_C, Theta_ref_SH_C)

    # (22)
    return e_dash_th_mid_C - 1

<div style="text-align: center;font-weight: bold;">表　記号に対応する変数</div>

<table>
<thead>
<tr>
<th colspan="2" rowspan="2" style="text-align: center;">算定式における記号</th>
<th colspan="4" style="text-align: center;">対応する変数</th>
</tr>
<tr>
<th>定格暖房能力<br>
運転時のヒート<br>
ポンプサイクル<br>
の理論効率<br>
$e_{th,rtd,H}$（式 19）<br>
を算定する場合</th>
<th>中間暖房能力<br>
運転時のヒート<br>
ポンプサイクル<br>
の理論効率<br>
$e_{th,mid,H}$（式 20）<br>
を算定する場合</th>
<th>定格冷房能力<br>
運転時のヒート<br>
ポンプサイクル<br>
の理論効率<br>
$e_{th,rtd,C}$（式 21）<br>
を算定する場合</th>
<th>中間冷房能力<br>
運転時のヒート<br>
ポンプサイクル<br>
の理論効率<br>
$e_{th,mid,C}$（式 22）<br>
を算定する場合</th>
</tr>
</thead>
<tbody>
<tr>
    <th>暖房時の蒸発温度<br>$\theta_{ref,env,H}$（式 24）<br>または<br>冷房時の凝縮温度<br>$\theta_{ref,cnd,C}$（式 27）</th>
    <th>外気温度(℃)$\theta_{ex}$</th>
    <td>7</td>
    <td>7</td>
    <td>35</td>
    <td>35</td>
</tr>
<tr>
    <th rowspan="5">暖房時の室内機熱交換器の<br>表面温度$\theta_{surf,hex,H}$（式 31）<br>または<br>冷房時の室内機熱交換器の<br>表面温度$\theta_{surf,hex,C}$（式 32）</th>
    <th>熱源機の入口における<br>空気温度(℃)$\theta_{hs,in}$</th>
    <td>20</td>
    <td>20</td>
    <td>27</td>
    <td>27</td>
</tr>
<tr>
    <th>熱源機の入口における<br>絶対湿度(kg/kg(DA))$X_{hs,in}$</th>
    <td>-</td>
    <td>-</td>
    <td>0.010376</td>
    <td>0.010376</td>
</tr>
<tr>
    <th>熱源機の風量($m^3$/h)$V_{hs,supply}$</th>
    <td>$V_{fan,rtd,H}$</td>
    <td>$V_{fan,mid,H}$</td>
    <td>$V_{fan,rtd,C}$</td>
    <td>$V_{fan,mid,C}$</td>
</tr>
<tr>
    <th>熱源機の暖房能力(W)$q_{hs,H}$</th>
    <td>$q_{hs,rtd,H}$</td>
    <td>$q_{hs,mid,H}$</td>
    <td>-</td>
    <td>-</td>
</tr>
<tr>
    <th>熱源機の冷房能力(W)$q_{hs,C}$</th>
    <td>-</td>
    <td>-</td>
    <td>$q_{hs,rtd,C}$</td>
    <td>$q_{hs,mid,C}$</td>
</tr>
</tbody>
</table>

#### A.4.4 冷媒温度

凝縮温度$\theta_{ref,cnd,H}$は、65℃を上回る場合、65℃に等しいとする。蒸発温度$\theta_{ref,evp,H}$は、-50℃を下回る場合、-50℃に等しいとし、また、$\theta_{ref,cnd,H}$ − 5.0を上回る場合、$\theta_{ref,cnd,H}$ −5.0に等しいとする。

$$\large
\theta_{ref,cnd,H}=\theta_{surf,hex,H}
$$

$$\large
\theta_{ref,evp,H}=\theta_{ex}-(0.100 \times \theta_{ref,cnd,H} + 2.95)
$$

$$\large
\theta_{ref,SC,H}=0.245 \times \theta_{ref,cnd,H} − 1.72
$$

$$\large
\theta_{ref,SH,H}=4.49 − 0.036  \times \theta_{ref,cnd,H}
$$

$\theta_{ex}$：外気温度(℃)  
$\theta_{ref,cnd,H}$：暖房時の冷媒の凝縮温度(℃)  
$\theta_{ref,evp,H}$：暖房時の冷媒の蒸発温度(℃)  
$\theta_{ref,SC,H}$：暖房時の冷媒の過冷却度(℃)  
$\theta_{ref,SH,H}$：暖房時の冷媒の過熱度(℃)  
$\theta_{surf,hex,H}$：暖房時の室内機熱交換器の表面温度(℃)  


In [91]:
def get_Theta_ref_cnd_H(Theta_sur_f_hex_H):
    """(23)

    :param Theta_sur_f_hex_H: 暖房時の室内機熱交換器の表面温度（℃）
    :return: 暖房時の冷媒の凝縮温度（℃）
    """
    Theta_ref_cnd_H = Theta_sur_f_hex_H

    if Theta_ref_cnd_H > 65:
        Theta_ref_cnd_H = 65

    return Theta_ref_cnd_H


def get_Theta_ref_evp_H(Theta_ex, Theta_ref_cnd_H):
    """(24)

    :param Theta_ex:外気温度（℃）
    :param Theta_ref_cnd_H:暖房時の冷媒の凝縮温度（℃）
    :return:暖房時の冷媒の蒸発温度（℃）
    """
    Theta_ref_evp_H = Theta_ex - (0.100 * Theta_ref_cnd_H + 2.95)

    if Theta_ref_evp_H < -50:
        Theta_ref_evp_H = -50

    if Theta_ref_evp_H > Theta_ref_cnd_H - 5.0:
        Theta_ref_evp_H = Theta_ref_cnd_H - 5.0

    return Theta_ref_evp_H


def get_Theta_ref_SC_H(Theta_ref_cnd_H):
    """(25)

    :param Theta_ref_cnd_H: 暖房時の冷媒の凝縮温度（℃）
    :return: 暖房時の冷媒の過冷却度（℃）
    """
    return 0.245 * Theta_ref_cnd_H - 1.72


def get_Theta_ref_SH_H(Theta_ref_cnd_H):
    """(26)

    :param Theta_ref_cnd_H: 暖房時の冷媒の凝縮温度（℃）
    :return: 暖房時の冷媒の過熱度（℃）
    """
    return 4.49 - 0.036 * Theta_ref_cnd_H

凝縮温度$\theta_{ref,cnd,C}$は、65℃を上回る場合、65℃に等しいとし、また、$\theta_{ref,evp,C}$ + 5.0を下回る場合、$\theta_{ref,evp,C}$ + 5.0に等しいとする。蒸発温度$\theta_{ref,evp,C}$は、-50℃を下回る場合、-50℃に等しいとする。

$$\large
\theta_{ref,cnd,C}=max(\theta_{ex}+27.4 −1.35 \times \theta_{ref,evp,C},\theta_{ex})
$$

$$\large
\theta_{ref,evp,C}=\theta_{surf,hex,C}
$$

$$\large
\theta_{ref,SC,C}=max(0.772 \times \theta_{ref,cnd,C} − 25.6 , 0)
$$

$$\large
\theta_{ref,SH,C}=max(0.194  \times \theta_{ref,cnd,C} −3.86 , 0)
$$

$\theta_{ex}$：外気温度(℃)  
$\theta_{ref,cnd,C}$：冷房時の冷媒の凝縮温度(℃)  
$\theta_{ref,evp,C}$：冷房時の冷媒の蒸発温度(℃)  
$\theta_{ref,SC,C}$：冷房時の冷媒の過冷却度(℃)  
$\theta_{ref,SH,C}$：冷房時の冷媒の過熱度(℃)  
$\theta_{surf,hex,C}$：冷房時の室内機熱交換器の表面温度(℃)  


In [92]:
def get_Theta_ref_cnd_C(Theta_ex, Theta_ref_evp_C):
    """(27)

    :param Theta_ref_evp_C: 冷房時の冷媒の蒸発温度（℃）
    :return: 冷房時の冷媒の凝縮温度（℃）
    """
    Theta_ref_cnd_C = np.maximum(Theta_ex + 27.4 - 1.35 * Theta_ref_evp_C, Theta_ex)

    if Theta_ref_cnd_C > 65:
        Theta_ref_cnd_C = 65

    if Theta_ref_cnd_C < Theta_ref_evp_C + 5.0:
        Theta_ref_cnd_C = Theta_ref_evp_C + 5.0

    return Theta_ref_cnd_C


def get_Theta_ref_evp_C(Theta_sur_f_hex_C):
    """(28)

    :param Theta_sur_f_hex_C:冷房時の室内機熱交換器の表面温度（℃）
    :return:冷房時の冷媒の蒸発温度（℃）
    """
    Theta_ref_evp_C = Theta_sur_f_hex_C

    if Theta_ref_evp_C < -50:
        Theta_ref_evp_C = -50

    return Theta_ref_evp_C


def get_Theta_ref_SC_C(Theta_ref_cnd_C):
    """(29)

    :param Theta_ref_cnd_H: 暖房時の冷媒の凝縮温度（℃）
    :return: 冷房時の冷媒の過冷却度（℃）
    """
    return np.maximum(0.772 * Theta_ref_cnd_C - 25.6, 0)


def get_Theta_ref_SH_C(Theta_ref_cnd_C):
    """(30)

    :param Theta_ref_cnd_H: 暖房時の冷媒の凝縮温度（℃）
    :return: 冷房時の冷媒の過熱度（℃）
    """
    return np.maximum(0.194 * Theta_ref_cnd_C - 3.86, 0)

### A.5 室内機熱交換器

#### A.5.1 熱交換器の表面温度

#### A.5.1.1 エネルギー消費量の算定における熱交換器の表面温度

$$\large
\theta_{surf,hex,H}=\frac{\theta_{hs,in}+\theta_{hs,out}}{2}+\frac{c_{p_{air}}\rho_{air}V_{hs,supply}(\theta_{hs,out}-\theta_{hs,in})\div 3600}{A_{e,hex}\alpha_{c,hex,H}}
$$

$$\large
\theta_{surf,hex,C}=\frac{\theta_{hs,in}+\theta_{hs,out}}{2}+\frac{c_{p_{air}}\rho_{air}V_{hs,supply}(\theta_{hs,in}-\theta_{hs,out})\div 3600}{A_{e,hex}\alpha_{c,hex,C}}
$$

$A_{e,hex}$：室内機熱交換器の表面積のうち熱交換に有効な面積($m^2$)  
$V_{hs,supply}$：熱源機の風量($m^3$/h)  
$\alpha_{c,hex,H}$：暖房時の室内熱交換器表面の顕熱伝達率(W/($m^2$・K))  
$\alpha_{c,hex,C}$：冷房時の室内熱交換器表面の顕熱伝達率(W/($m^2$・K))  
$\theta_{hs,in}$：熱源機の入口における空気温度(℃)  
$\theta_{hs,out}$：熱源機の出口における空気温度(℃)  
$\theta_{surf,hex,C}$：冷房時の室内機熱交換器の表面温度(℃)  
$\theta_{surf,hex,H}$：暖房時の室内機熱交換器の表面温度(℃)  
$c_{p_{air}}$：空気の比熱(J/(kg・K))  
$\rho_{air}$：空気の密度(kg/$m^3$)  


In [93]:
def get_Theta_sur_f_hex_H_calc(Theta_hs_in, Theta_hs_out, V_hs_supply, alpha_c_hex_H):
    """(31)

    :param Theta_hs_in: 熱源機の入口における空気温度（℃）
    :param Theta_hs_out: 熱源機の出口における空気温度（℃）
    :param V_hs_supply: 熱源機の風量（m3/h）
    :param alpha_c_hex_H: 暖房時の室内熱交換器表面の顕熱伝達率（W/(m2・K)）
    :return: エネルギー消費量の算定における熱交換器の表面温度を算定する場合の暖房時の室内機熱交換器の表面温度（℃）
    """
    c_p_air = get_c_p_air()
    rho_air = get_rho_air()
    A_e_hex = get_A_e_hex()

    Theta_sur_f_hex_H = ((Theta_hs_in + Theta_hs_out) / 2) \
                        + (c_p_air * rho_air * V_hs_supply * (Theta_hs_out - Theta_hs_in) / 3600) / (A_e_hex * alpha_c_hex_H)

    return Theta_sur_f_hex_H


def get_Theta_sur_f_hex_C_calc(Theta_hs_in, Theta_hs_out, V_hs_supply, alpha_c_hex_C):
    """(32)

    :param Theta_hs_in: 熱源機の入口における空気温度（℃）
    :param Theta_hs_out: 熱源機の出口における空気温度（℃）
    :param V_hs_supply: 熱源機の風量（m3/h）
    :param alpha_c_hex_C: 冷房時の室内熱交換器表面の顕熱伝達率（W/(m2・K)）
    :return: エネルギー消費量の算定における熱交換器の表面温度を算定する場合の冷房時の室内機熱交換器の表面温度（℃）
    """
    c_p_air = get_c_p_air()
    rho_air = get_rho_air()
    A_e_hex = get_A_e_hex()

    Theta_sur_f_hex_C = ((Theta_hs_in + Theta_hs_out) / 2) \
                        - (c_p_air * rho_air * V_hs_supply * (Theta_hs_in - Theta_hs_out) / 3600) / (A_e_hex * alpha_c_hex_C)

    return Theta_sur_f_hex_C

#### A.5.1.2 JIS 試験における熱交換器の表面温度

$$\large
\theta_{surf,hex,H}=\theta_{hs,in} + \frac{q_{hs,H}}{2c_{p_{air}}\rho_{air}V_{hs,supply}}\times3600+\frac{q_{hs,H}}{A_{e,hex}\alpha_{c,hex,H}}
$$

$A_{e,hex}$：室内機熱交換器の表面積のうち熱交換に有効な面積($m^2$)  
$V_{hs,supply}$：熱源機の風量($m^3$/h)  
$q_{hs,H}$：熱源機の暖房能力(W)  
$\alpha_{c,hex,H}$：暖房時の室内熱交換器表面の顕熱伝達率(W/($m^2$・K))  
$\theta_{hs,in}$：熱源機の入口における空気温度(℃)  
$\theta_{surf,hex,H}$：暖房時の室内機熱交換器の表面温度(℃)  
$c_{p_{air}}$：空気の比熱(J/(kg・K))  
$\rho_{air}$：空気の密度(kg/$m^3$)  


In [94]:
def get_Theta_sur_f_hex_H_JIS(V_fan_x_H, q_hs_X_H, alpha_c_hex_H):
    """(33)

    :param V_fan_x_H: 熱源機の風量（m3/h）※式(19)を算定する場合はV_fan_rtd_H、式(20)を算定する場合はV_fan_mid_H
    :param q_hs_X_H: 熱源機の暖房能力（W）※式(19)を算定する場合はq_hs_rtd_H、式(20)を算定する場合はq_hs_mid_H
    :param alpha_c_hex_H: 暖房時の室内機熱交換器の表面温度（℃）
    :return: JIS試験における熱交換器の表面温度を算定する場合の暖房時の室内機熱交換器の表面温度（℃）
    """
    # 表5より
    Theta_hs_in = 20
    V_hs_supply = V_fan_x_H
    q_hs_H = q_hs_X_H

    c_p_air = get_c_p_air()
    rho_air = get_rho_air()
    A_e_hex = get_A_e_hex()

    Theta_sur_f_hex_H_JIS = Theta_hs_in + (q_hs_H / (2 * c_p_air * rho_air * V_hs_supply)) * 3600 \
                            + (q_hs_H / (A_e_hex * alpha_c_hex_H))

    return Theta_sur_f_hex_H_JIS

$$\large
q_{hs,C}=q_{hs,CS}+q_{hs,CL}
$$

$$\large
q_{hs,CS}=(\theta_{hs,in}-\theta_{surf,hex,C})\div \left( \frac{3600}{2c_{p_{air}}\rho_{air}V_{hs,supply}} + \frac{1}{A_{e,hex}\alpha_{c,hex,H}} \right)
$$

$$\large
q_{hs,CL}= max \left( (X_{hs,in}-X_{surf,hex,C})\div \left( \frac{3600}{2L_{wtr}\rho_{air}V_{hs,supply}\times10^3} + \frac{1}{L_{wtr}A_{e,hex}\alpha'_{c,hex,H}\times10^3} \right) , 0 \right)
$$

$A_{e,hex}$：室内機熱交換器の表面積のうち熱交換に有効な面積($m^2$)  
$q_{hs,C}$：熱源機の冷房能力(W)  
$q_{hs,CS}$：熱源機のの冷房顕熱能力(W)  
$q_{hs,CL}$：熱源機の冷房潜熱能力(W)  
$V_{hs,supply}$：熱源機の風量($m^3$/h)  
$X_{hs,in}$：熱源機の入口における絶対湿度(kg/kg(DA))  
$X_{surf,hex,C}$：冷房時の室内機熱交換器の表面絶対湿度(kg/kg(DA))  
$\alpha_{c,hex,C}$：冷房時の室内熱交換器表面の顕熱伝達率(W/($m^2$・K))  
$\alpha'_{c,hex,C}$：冷房時の室内熱交換器表面の潜熱伝達率(kg/($m^2$・s))  
$\theta_{hs,in}$：熱源機の入口における空気温度(℃)  
$\theta_{surf,hex,C}$：冷房時の室内機熱交換器の表面温度(℃)  
$c_{p_{air}}$：空気の比熱(J/(kg・K))  
$L_{wtr}$：水の蒸発潜熱(kJ/kg)  
$\rho_{air}$：空気の密度(kg/$m^3$)  


In [95]:
def calc_Theta_sur_f_hex_C_JIS(Theta_surf_hex_C, V_fan_x_C, alpha_c_hex_C, alpha_dash_c_hex_C):
    """(34a)

    :param Theta_surf_hex_C:冷房時の室内機熱交換器の表面温度（℃）
    :param V_fan_x_C:熱源機の風量（m3/h）
    :param alpha_c_hex_C:冷房時の室内熱交換器表面の顕熱伝達率（W/(m2・K)）
    :param alpha_dash_c_hex_C:冷房時の室内熱交換器表面の潜熱伝達率（kg/(m2・s)）
    :return:冷房時の室内機熱交換器の表面温度（℃） ※連立方程式の解
    """
    # (34b)
    q_hs_CS = get_q_hs_CS(Theta_surf_hex_C, V_fan_x_C, alpha_c_hex_C)

    # (34c)
    q_hs_CL = get_q_hs_CL(Theta_surf_hex_C, V_fan_x_C, alpha_dash_c_hex_C)

    # (34a)
    return q_hs_CS + q_hs_CL


def get_q_hs_CS(Theta_surf_hex_C, V_fan_x_C, alpha_c_hex_C):
    """(34b)

    :param Theta_surf_hex_C: 冷房時の室内機熱交換器の表面温度（℃）
    :param V_fan_x_C: 熱源機の風量（m3/h）
    :param alpha_c_hex_C: 冷房時の室内熱交換器表面の顕熱伝達率（W/(m2・K)）
    :return:熱源機の冷房顕熱能力（W）
    """
    # 表5より
    Theta_hs_in = 27
    V_hs_supply = V_fan_x_C

    c_p_air = get_c_p_air()
    rho_air = get_rho_air()
    A_e_hex = get_A_e_hex()

    return (Theta_hs_in - Theta_surf_hex_C) / (3600 / (2 * c_p_air * rho_air * V_hs_supply) + 1 / (A_e_hex * alpha_c_hex_C))


def get_q_hs_CL(Theta_surf_hex_C, V_fan_x_C, alpha_dash_c_hex_C):
    """(34c)

    :param Theta_surf_hex_C: 冷房時の室内機熱交換器の表面温度（℃）
    :param V_fan_x_C: 熱源機の風量（m3/h）
    :param alpha_dash_c_hex_C: 冷房時の室内熱交換器表面の潜熱伝達率（kg/(m2・s)）
    :return: 熱源機の冷房潜熱能力（W）
    """
    # 表5より
    X_hs_in = 0.010376
    V_hs_supply = V_fan_x_C

    L_wtr = get_L_wtr()
    rho_air = get_rho_air()
    A_e_hex = get_A_e_hex()

    # ------ここから第11章5節------

    Theta = Theta_surf_hex_C

    # 大気圧（Pa）
    F = 101325

    # 絶対温度（K）
    T = Theta + 273.16 #(6)

    # 表1 式(5b)における係数の値
    a1 = -6096.9385
    a2 = 21.2409642
    a3 = -0.02711193
    a4 = 0.00001673952
    a5 = 2.433502
    b1 = -6024.5282
    b2 = 29.32707
    b3 = 0.010613863
    b4 = -0.000013198825
    b5 = -0.49382577

    #(5b)
    if Theta > 0:
        k = a1 / T + a2 + a3 * T + a4 * T ** 2 + a5 * np.log(T)
    else:
        k = b1 / T + b2 + b3 * T + b4 * T ** 2 + b5 * np.log(T)

    # 飽和水蒸気圧（Pa）
    P_vs = np.exp(k) #(5a)

    # 飽和空気の絶対湿度（kg/kg(DA)）
    X_s = 0.622 * (P_vs / (F - P_vs)) #(3)

    # ------ここまで第11章5節------

    X_surf_hex_C = X_s

    # (34c)
    a = (X_hs_in - X_surf_hex_C) / (3600 / (2 * L_wtr * rho_air * V_hs_supply * 10 ** 3) \
                                    + 1 / (L_wtr * A_e_hex * alpha_dash_c_hex_C * 10 ** 3))
    return np.maximum(a, 0)

#### A.5.2 熱交換器表面の顕熱伝達率

暖房時：

$$\large
\alpha_{c,hex,H}=\left\{ -0.0017 \times \left( \frac{V_{hs,supply}\div3600}{A_{f,hex}} \right)^2 + 0.044 \times \frac{V_{hs,supply}\div3600}{A_{f,hex}} + 0.00271  \right\} \times 10^3
$$

冷房時：

$$\large
\alpha_{c,hex,C}=\alpha'_{c,hex,C}\times(c_{p_{air}}+c_{p_{w}}X_{hs,in})
$$

$$\large
\alpha'_{c,hex,C}=0.050 \times ln\left( \frac{max(V_{hs,supply},400)\div 3600}{A_{f,hex}} \right) + 0.073
$$

$A_{f,hex}$：室内機熱交換器の全面面積のうち熱交換に有効な面積($m^2$)  
$V_{hs,supply}$：熱源機の風量($m^3$/h)  
$X_{hs,in}$：熱源機の入口における絶対湿度(kg/kg(DA))  
$\alpha_{c,hex,H}$：暖房時の室内熱交換器表面の顕熱伝達率(W/($m^2$・K))  
$\alpha_{c,hex,C}$：冷房時の室内熱交換器表面の顕熱伝達率(W/($m^2$・K))  
$\alpha'_{c,hex,C}$：冷房時の室内熱交換器表面の潜熱伝達率(kg/($m^2$・s))  
$c_{p_{air}}$：空気の比熱(J/(kg・K))  
$c_{p_{w}}$：水蒸気の比熱(J/(kg・K))  


In [96]:
def get_alpha_c_hex_H(V_fan_x_H):
    """(35)

    :param V_fan_x_H:熱源機の風量（m3/h）
    :return:暖房時の室内熱交換器表面の顕熱伝達率（W/(m2・K)）
    """
    # 表5より
    V_hs_supply = V_fan_x_H

    A_f_hex = get_A_f_hex()

    alpha_c_hex_H = (-0.0017 * ((V_hs_supply / 3600) / A_f_hex) ** 2 \
                     + 0.044 * ((V_hs_supply / 3600) / A_f_hex) + 0.0271) * 10 ** 3

    return alpha_c_hex_H


def get_alpha_c_hex_C(V_fan_x_C, X_hs_in):
    """(36)

    :param V_fan_x_C:熱源機の風量（m3/h）
    :param X_hs_in:冷房時の熱源機の入口における絶対湿度（kg/kg(DA)）
    :return:冷房時の室内熱交換器表面の顕熱伝達率（W/(m2・K)）および 冷房時の室内熱交換器表面の潜熱伝達率（kg/(m2・s)）
    """
    # 表5より
    V_hs_supply = V_fan_x_C

    A_f_hex = get_A_f_hex()
    c_p_air = get_c_p_air()
    c_p_w = get_c_p_w()

    a = np.clip(V_hs_supply, 400, None)

    # (36b)
    alpha_dash_c_hex_C = 0.050 * np.log((a / 3600) / A_f_hex) + 0.073

    # (36a)
    alpha_c_hex_C = alpha_dash_c_hex_C * (c_p_air + c_p_w * X_hs_in)

    return alpha_c_hex_C, alpha_dash_c_hex_C

#### A.5.3 熱交換器の表面積

室内機熱交換器の全面面積のうち熱交換に有効な面積$A_{f,hex}$は、0.23559 m2 とする。
室内機熱交換器の表面積のうち熱交換に有効な面積$A_{e,hex}$は、6.396 m2 とする。

In [97]:
# 室内機熱交換器の全面面積のうち熱交換に有効な面積 (m2)
def get_A_f_hex():
    return 0.23559


# 室内機熱交換器の表面積のうち熱交換に有効な面積 (m2)
def get_A_e_hex():
    return 6.396

### A.6 送風機

$$\large
E_{E,fan,H,d,t}=\left\{ \begin{array}{ll}
    max\left((P_{fan,rtd,H}-f_{SFP}V_{hs,vent,d,t}) \times \frac{V_{hs,supply,d,t}-V_{hs,vent,d,t}}{V_{hs,dsgn,H}-V_{hs,vent,d,t}} \times 10^{-3}, 0 \right) &(q_{hs,H,d,t} > 0)\\ 
    0 &(q_{hs,H,d,t} \leq 0)\\ 
    \end{array}
\right.
$$

$$\large
E_{E,fan,C,d,t}=\left\{ \begin{array}{ll}
    max\left((P_{fan,rtd,C}-f_{SFP}V_{hs,vent,d,t}) \times \frac{V_{hs,supply,d,t}-V_{hs,vent,d,t}}{V_{hs,dsgn,C}-V_{hs,vent,d,t}} \times 10^{-3}, 0 \right) &(q_{hs,C,d,t} > 0)\\ 
    0 &(q_{hs,C,d,t} \leq 0)\\ 
    \end{array}
\right.
$$

$E_{E,fan,C,d,t}$：日付$d$の時刻$t$における1 時間当たりの送風機の消費電力量のうちの冷房設備への付加分(kWh/h)  
$E_{E,fan,H,d,t}$：日付$d$の時刻$t$における1 時間当たりの送風機の消費電力量のうちの暖房設備への付加分(kWh/h)  
$f_{SFP}$：全般換気設備の比消費電力(W/($m^3$/h))  
$P_{fan,rtd,C}$：定格冷房能力運転時の送風機の消費電力(W)  
$P_{fan,rtd,H}$：定格暖房能力運転時の送風機の消費電力(W)  
$q_{hs,C,d,t}$：日付$d$の時刻$t$における1 時間当たりの熱源機の平均冷房能力(W)  
$q_{hs,H,d,t}$：日付$d$の時刻$t$における1 時間当たりの熱源機の平均暖房能力(W)  
$V_{hs,dsgn,C}$：冷房時の設計風量($m^3$/h)  
$V_{hs,dsgn,H}$：暖房時の設計風量($m^3$/h)  
$V_{hs,vent,d,t}$：日付$d$の時刻$t$における熱源機の風量のうちの全般換気分($m^3$/h)  


In [98]:
def get_E_E_fan_H_d_t(P_fan_rtd_H, V_hs_vent_d_t, V_hs_supply_d_t, V_hs_dsgn_H, q_hs_H_d_t):
    """(37)

    :param P_fan_rtd_H:定格暖房能力運転時の送風機の消費電力（W）
    :param V_hs_vent_d_t:日付dの時刻tにおける熱源機の風量のうちの全般換気分（m3/h）
    :param V_hs_supply_d_t:
    :param V_hs_dsgn_H:暖房時の設計風量（m3/h）
    :param q_hs_H_d_t:日付dの時刻tにおける1時間当たりの熱源機の平均暖房能力（-）
    :return:日付dの時刻tにおける1時間当たりの送風機の消費電力量のうちの暖房設備への付加分（kWh/h）
    """
    f_SFP = get_f_SFP()
    E_E_fan_H_d_t = np.zeros(24 * 365)

    a = (P_fan_rtd_H - f_SFP * V_hs_vent_d_t) \
        * ((V_hs_supply_d_t - V_hs_vent_d_t) / (V_hs_dsgn_H - V_hs_vent_d_t)) * 10 ** (-3)

    E_E_fan_H_d_t[q_hs_H_d_t > 0] = np.clip(a[q_hs_H_d_t > 0], 0, None)

    return E_E_fan_H_d_t

def get_e_rtd_H():
    """定格暖房エネルギー消費効率
    
    :return: 定格暖房エネルギー消費効率
    :rtype: float
    """
    # 定格暖房エネルギー消費効率
    e_rtd_H = 3.76
    return e_rtd_H

def get_e_rtd_C():
    """定格冷房エネルギー消費効率
    
    :return: 定格冷房エネルギー消費効率
    :rtype: float
    """
    # 定格冷房エネルギー消費効率
    e_rtd_C = 3.17
    return e_rtd_C

def get_E_E_fan_C_d_t(P_fan_rtd_C, V_hs_vent_d_t, V_hs_supply_d_t, V_hs_dsgn_C, q_hs_C_d_t):
    """(38)

    :param P_fan_rtd_C:定格冷房能力運転時の送風機の消費電力（W）
    :param V_hs_vent_d_t:日付dの時刻tにおける熱源機の風量のうちの全般換気分（m3/h）
    :param V_hs_supply_d_t:
    :param V_hs_dsgn_C:冷房時の設計風量（m3/h）
    :param q_hs_C_d_t:日付dの時刻tにおける1時間当たりの熱源機の平均冷房能力（-）
    :return:日付dの時刻tにおける1時間当たりの送風機の消費電力量のうちの暖房設備への付加分（kWh/h）
    """
    f_SFP = get_f_SFP()
    E_E_fan_C_d_t = np.zeros(24 * 365)

    a = (P_fan_rtd_C - f_SFP * V_hs_vent_d_t) \
        * ((V_hs_supply_d_t - V_hs_vent_d_t) / (V_hs_dsgn_C - V_hs_vent_d_t)) * 10 ** (-3)

    E_E_fan_C_d_t[q_hs_C_d_t > 0] = np.clip(a[q_hs_C_d_t > 0], 0, None)

    return E_E_fan_C_d_t

全般換気設備の比消費電力$f_{SFP}$は、ダクト式第二種換気設備（直流）として 0.144（= 0.4 × 0.36）とする。

In [99]:
# 全般換気設備の比消費電力（W/(m3/h)）
def get_f_SFP():
    return 0.4 * 0.36

### A.7 熱源機および送風機の仕様

省略

### A.8 空気・水蒸気・水の物性値

空気の比熱$c_{p_{air}}$は、1006.0 J/(kg・K)とする。空気の密度$\rho_{air}$は、1.2 kg/$m^3$ とする。
水蒸気の定圧比熱$c_{p_{w}}$は、1.846 J/(kg・K)とする。

$$\large
L_{wtr}=2500.8 −2.3668 \times \theta
$$

$L_{wtr}$：水の蒸発潜熱(kJ/kg)  
$\theta$：温度(℃)  


In [100]:
# 空気の比熱 (J/Kg・K)
def get_c_p_air():
    return 1006.0


# 空気の密度 (kg/m3)
def get_rho_air():
    return 1.2


# 水蒸気の定圧比熱  (J/Kg・K)
def get_c_p_w():
    return 1.846


# 水の蒸発潜熱 (kJ/kg) (39)
def get_L_wtr():
    Theta = get_Theta()
    return 2500.8 - 2.3668 * Theta

温度$\theta$は、冷房時を仮定し、27℃とする。


In [101]:
# 冷房時を仮定した温度 (℃)
def get_Theta():
    return 27